In [3]:
from sklearn.cluster import DBSCAN
import pandas as pd

In [4]:
df = pd.read_csv("dynamicduos.csv")

In [5]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Rk,Player,Pos,Age,Tm,G,MP,PER,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player-additional
0,2,2,3,"Adebayo, Bam",C,24,MIA,56,1825,21.8,...,3.6,3.5,7.2,0.188,NaN,1.7,2.1,3.8,2.7,adebaba01
1,15,15,12,"Antetokounmpo, Giannis",PF,27,MIL,67,2204,32.1,...,9.2,3.7,12.9,0.281,NaN,7.6,3.5,11.2,7.4,antetgi01
2,49,49,42,"Beasley, Malik",SG,25,MIN,79,1976,11.9,...,1.7,1.2,2.9,0.069,NaN,0.7,-1.5,-0.7,0.7,beaslma01
3,99,99,80,"Brunson, Jalen",SG,25,DAL,79,2524,17.1,...,4.9,2.7,7.5,0.143,NaN,0.9,0.0,0.9,1.9,brunsja01
4,109,109,90,"Campazzo, Facundo",PG,30,DEN,65,1184,11.1,...,0.6,1.3,2.0,0.081,NaN,-2.1,1.3,-0.8,0.4,campafa01


In [6]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Rk', 'Player', 'Pos', 'Age', 'Tm', 'G',
       'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'Unnamed: 19', 'OWS', 'DWS', 'WS',
       'WS/48', 'Unnamed: 24', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Player-additional'],
      dtype='object')

In [7]:
new_df = df[['MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]

In [8]:
X_train = new_df.fillna(new_df.mean())

In [9]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from kneed import KneeLocator

scaler = StandardScaler()
scaled_features = scaler.fit_transform(X_train)


kmeans_kwargs = {
 "init": "random",
 "n_init": 10,
   "max_iter": 300,
    "random_state": 42,
}

sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)


In [10]:
kl = KneeLocator(
    range(1, 11), sse, curve="convex", direction="decreasing"
)


In [11]:
kl.elbow

5

In [12]:
kmeans_kwargs = {
 "init": "random",
 "n_init": 10,
   "max_iter": 300,
    "random_state": 42,
}
kmeans = KMeans(n_clusters=5, **kmeans_kwargs)
kmeans.fit(scaled_features)

KMeans(init='random', n_clusters=5, n_init=10, random_state=42)

In [13]:
df.loc[:, 'Cluster'] = kmeans.labels_

In [14]:
df = df[['Player','Pos', 'MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Cluster']]

In [15]:
df = df.replace(['SG-PG', 'SG-SF', 'SF-SG', 'PF-SF',
       'C-PF', 'SG-PG-SF', 'PG-SG'], ['SG', 'SG', 'SF', 'PF', 'C', 'SG', 'PG'])
df['Pos'].value_counts()

PG    18
SG    10
PF     4
C      1
SF     1
Name: Pos, dtype: int64

In [38]:
df[df.Cluster==4]['Pos'].value_counts()
#0 and 2 - More than half of all high performing players in duos, supportive and agile guards but on par underperforming
#1 -> Extremely high efficiency bigs
#2 and 3-> High value (VORP) players with very high AST ratios and strong shooting
#4 -> Point guards with high offensive win shares

PG    1
Name: Pos, dtype: int64

In [39]:
duos = pd.read_csv("duos.csv")

In [52]:
def c_process(p1, p2):
    #print(p1, p2)
    return str(df[df.Player == p1]['Cluster'].tolist()[0]) + "-" + str(df[df.Player==p2]['Cluster'].tolist()[0])

In [55]:
duos['ClusterVals'] = duos.apply(lambda row : c_process(row['Player 1'],
                     row['Player 2']), axis = 1)

Joe, Isaiah Harden, James
Adebayo, Bam Lowry, Kyle
Pritchard, Payton Nesmith, Aaron
Antetokounmpo, Giannis Smart, Javonte
Flynn, Malachi Siakam, Pascal
DeRozan, DeMar Dotson, Devon
Young, Trae Stephenson, Lance
Culver, Jarrett Jones, Tyus
Curry, Stephen Green, Draymond
Brunson, Jalen Ntilikina, Frank
Mitchell, Donovan Conley, Mike
Campazzo, Facundo Hyland, Bones
Beasley, Malik Nowell, Jaylen
Harper, Jared Temple, Garrett


In [58]:
duos['ClusterVals'].value_counts()

0-0    4
3-0    3
0-3    2
3-3    2
1-3    1
1-2    1
4-0    1
Name: ClusterVals, dtype: int64

In [ ]:
#00
#03
#33